In [ ]:
from requests import request, Session
from pandas.io.json import json_normalize
import pandas as pd
#from requests.exceptions import ConnectionError, Timeout, TooManyRedirect
import json
import os
url = 'https://pro-api.coinmarketcap.com/v1/cryptocurrency/listings/latest'
#Original Sandbox Environment: 'https://sandbox-api.coinmarketcap.com/v1/cryptocurrency/listings/latest'
parameters = {
    'start':'1',
    'limit':'15',
    'convert':'USD'
}
headers = {
    'accepts':'application/json',
    'X-CMC_PRO_API_KEY':'0ad53085-1cb2-4eb8-ad9e-3ffbd7e56509',   
}
session = Session()
session.headers.update(headers)

try:
    response = session.get(url, params=parameters)
    data = json.loads(response.text)
#Print data
except (ConnectionError, Timeout, TooManyRedirects) as e:
    print(e)

        



In [ ]:
#Checking for data type
type(data)

In [ ]:
import pandas as pd

#This allows to see all the columns, not just like 15
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows',None)
print(data)

In [ ]:
#Normalizing the data and making it look pretty in the dataframe
df = pd.json_normalize(data['data'])
df['timestamp'] = pd.to_datetime('now')
print(df)

In [ ]:
#Initiating a function
def api_runner():
    global df
    url = 'https://pro-api.coinmarketcap.com/v1/cryptocurrency/listings/latest'
    #Original Sandbox Environment: 'https://pro-api.coinmarketcap.com/v1/cryptocurrency/listings/latest'
    parameters = {
        'start':'1',
        'limit':'15',
        'convert':'USD'        
    }
    headers = {
        'accept': 'application/json',
        'X-CMC_PRO_API_KEY': '0ad53085-1cb2-4eb8-ad9e-3ffbd7e56509',
    }
    session = Session()
    session.headers.update(headers)
    
    try:
        response = session.get(url, params=parameters)
        data = json.loads(response.text)
            #Print data
    except (ConnectionError, Timeout, TooManyRedirects) as e:
            print(e)
#NOTE:
# I had to go in and put "jupyter notebook --NotebookApp.iopub_data_rate_limit=1e10"
# Into the Anaconda Prompt to change this to allow to pull data
#Using a dataframe
df2 = pd.json_normalize(data['data'])
df2['Timestamp'] = pd.to_datetime('now')
df = df.append(df2)
    
    
#Creating a csv file and appending data to it
df = pd.json_normalize(data['data'])
df['timestamp'] = pd.to_datetime('now')
df
    
if not os.path.isfile(r'C:\Users\USER\OneDrive\ANALYSIS\API.csv'):
    df.to_csv(r'C:\Users\USER\OneDrive\ANALYSIS\API.csv',header = 'column_names')
else:
    df.to_csv(r'C:\Users\USER\OneDrive\ANALYSIS\API.csv', mode='a', header = False)
        
# Then to read in the file: 
#df = pd.read_csv(r'C:\Users\USER\OneDrive - Diamonds\API.csv')        


In [ ]:
import os
from time import time
from time import sleep

for i in range(333):
    api_runner()
    print('API Runner Completed')
    sleep(60) # Sleep for 60 seconds
exit()


In [ ]:
df72 = pd.read_csv(r'C:\Users\USER\OneDrive\ANALYSIS\API.csv')
df72

In [ ]:
df

In [ ]:
pd.set_option('display.float_format', lambda x: '%.5f' % x)


In [ ]:
df

In [ ]:
# Looking at the coin trends over time
df3 = df.groupby('name', sort=False)[['quote.USD.percent_change_1h',
                                      'quote.USD.percent_change_24h',
                                      'quote.USD.percent_change_7d',
                                      'quote.USD.percent_change_30d',
                                      'quote.USD.percent_change_60d',
                                      'quote.USD.percent_change_90d']].mean()
df3


















In [ ]:
df4 = df3.stack()
df4

In [ ]:
type(df4)

In [ ]:
df5 = df4.to_frame(name='values')
df5

In [ ]:
df.count()

In [ ]:
#Because of how it's structured above we need to set an index. I don't want to pass a column as an index for this dataframe
#So I'm going to create a range and pass that as the dataframe. You can make this more dynamic, but I'm just going to hard code it

index = pd.Index(range(90))
# Set the aboce dataframe index objext as the index
#Using set_index() function
df6 = df5.reset_index()
df6

# If it only has the index and values try doing reset_index like "df5.reset_index()"

In [ ]:
df7 = df6.rename(columns={'level_1': 'Time_change'})
df7


In [ ]:
#Reformatting the Time change Column
df7['Time_change'] = df7['Time_change'].replace(['quote.USD.percent_change_1h','quote.USD.percent_change_24h','quote.USD.percent_change_7d','quote.USD.percent_change_30d','quote.USD.percent_change_60d','quote.USD.percent_change_90d'],['1h','24h','7d','30d', '60d', '90d'])
print(df7)



In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
p = sns.catplot(x='Time_change', y='values', hue='name', data=df7, kind='point')
p

In [ ]:
df10=df[['name', 'quote.USD.price','timestamp']]
#df10=df10.query("name == 'Bitcoin'")
df10